In [ ]:
Packages <- c("modelsummary", "lmerTest", "arrow", "mgcv", "dplyr" , "effects")
invisible(lapply(Packages, library, character.only = TRUE))

In [ ]:
datafolder <- '/home/hoyinchan/blue/Data/data2022/'

In [ ]:
loincs_cofounder <- c('LG6033-7', #AST       
                    'LG49755-8') #uric acid
loincs_dependents <- c('LG49864-8', #ca
                     'LG11363-5', #Na
                     'LG49936-4', 'LG6657-3') #K

In [ ]:
K_Bound <-  c(3.2, 4.4) # 2823-3
Na_Bound <- c(132, 144) # 2951-2
Ca_Bound <- c(8, 9.3) # 17861-6

In [ ]:
path1 = paste0(datafolder,'confounder_regression.parquet')
df <- arrow::read_parquet(path1)

In [ ]:
df <- df %>%  mutate_at(vars('if_confound_med_LNa', 'if_confound_med_HCa', 'if_confound_med_HK', 'if_confound_med_LK', 'low_K', 'high_K', 'low_Na', 'high_Na', 'low_Ca', 'high_Ca', 'ID2'), factor)

In [ ]:
df2 <- df

In [ ]:
calculate_slope <- function(df, chem, Na_Bound, K_Bound, Ca_Bound, last_v=NULL, raw_v=NULL, med_v=NULL){
    df_filter <- df
    df_filter_con <- df
    if (chem == 'Na'){ 
        eqn <- sCr ~  (1|ID2) + Na
        eqn_con <- sCr ~  (1|ID2) + Na    
        if (last_v == 'L'){
            df_filter <- df_filter %>% filter(low_Na==1)
        }
        if (raw_v == 'L'){
            df_filter <- df_filter %>% filter(Na<=Na_Bound[1])
        }
        df_filter_con <- df_filter %>% filter(if_confound_med_LNa==0)
    }
    
    if (chem == 'Ca'){
        eqn <- sCr ~  (1|ID2) + Ca
        eqn_con <- sCr ~  (1|ID2) + Ca 
        if (last_v == 'H'){
            df_filter <- df_filter %>% filter(high_Ca==1)
        }
        if (raw_v == 'H'){
            df_filter <- df_filter %>% filter(Ca>=Ca_Bound[2])
        }
        df_filter_con <- df_filter %>% filter(if_confound_med_HCa==0)        
    }
    
    if (chem == 'K'){
        if ((last_v != 'L') | (raw_v != 'L')){
            eqn <- sCr ~  (1|ID2) + K
#            eqn_con <- sCr ~  (1|ID2) + K + AST + UA
#            eqn_con <- sCr ~  (1|ID2) + K + UA +AST
            eqn_con <- sCr ~  (1|ID2) + K + UA            
        } else {
            eqn <- sCr ~  (1|ID2) + K
            eqn_con <- sCr ~  (1|ID2) + K
        }
        
        if (last_v == 'L'){
            df_filter <- df_filter %>% filter(low_K==1)
            df_filter_con <- df_filter %>% filter(if_confound_med_LK==0)                       
        }
        if (raw_v == 'L'){
            df_filter <- df_filter %>% filter(K<=K_Bound[1])
            df_filter_con <- df_filter %>% filter(if_confound_med_LK==0)           
        }
        
        if (last_v == 'H'){
            df_filter <- df_filter %>% filter(high_K==1)
            df_filter_con <- df_filter %>% filter(if_confound_med_HK==0)                       
        }
        if (raw_v == 'H'){
            df_filter <- df_filter %>% filter(K>=K_Bound[2])
            df_filter_con <- df_filter %>% filter(if_confound_med_HK==0)           
        }
    }
        
    x_con <- lmer(eqn_con, data=df_filter_con, REML = FALSE, control = lmerControl(optimizer ='optimx', optCtrl=list(method='L-BFGS-B')))    
    x <-     lmer(eqn,     data=df_filter,     REML = FALSE, control = lmerControl(optimizer ='optimx', optCtrl=list(method='L-BFGS-B'))) 
    slope_con <- fixef(x_con)[[chem]]
    slope     <- fixef(x)[[chem]]    
    p_con     <- summary(x_con)$coefficients[chem, "Pr(>|t|)"]
    p         <- summary(x)$coefficients[chem, "Pr(>|t|)"]    
    diff      <- (slope-slope_con)/slope*100
    
    return (c(round(c(slope, slope_con, p<=0.05, p_con<=0.05, diff),4), x, x_con))
}
        

In [ ]:
results = calculate_slope(df2, 'K', Na_Bound, K_Bound, Ca_Bound, last_v='L', raw_v='A')    
results[1:5]

In [ ]:
results = calculate_slope(df2, 'K', Na_Bound, K_Bound, Ca_Bound, last_v='L', raw_v='A')    
results[1:5]

In [ ]:
results = calculate_slope(df2, 'K', Na_Bound, K_Bound, Ca_Bound, last_v='H', raw_v='A')    
results[1:5]

In [ ]:
results <- calculate_slope(df, 'Na', Na_Bound, K_Bound, Ca_Bound, last_v='L', raw_v='A')    
results[1:5]

In [ ]:
results <- calculate_slope(df, 'Ca', Na_Bound, K_Bound, Ca_Bound, last_v='H', raw_v='A')    
results[1:5]